In [ ]:
import geopandas as gpd

In [ ]:
gdf=gpd.read_file('data/grilla_mdt_urbano_epsg4326.geojson')

In [ ]:
gdf.plot()

In [ ]:
gdf

In [ ]:
gdf.info()

# 함경도 신흥

In [5]:
import ee
import geemap

In [6]:
geemap.ee_initialize()

In [7]:
Map = geemap.Map(center=[42.435677739012725, 129.74609161652995], zoom=12)
Map

Map(center=[42.435677739012725, 129.74609161652995], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-74.7222, -8.5867, -74.1596, -8.2824)
    Map.addLayer(roi)
    Map.centerObject(roi)

In [ ]:
timelapse = geemap.landsat_timelapse(
    roi,
    out_gif='north_korea_huirong.gif',
    start_year=1984,
    end_year=2022,
    start_date='01-01',
    end_date='12-31',
    bands=['SWIR1', 'NIR', 'Red'],
    frames_per_second=5,
    title='Landsat Timelapse',
    progress_bar_color='blue',
    mp4=True,
)
geemap.show_image(timelapse)

# Landcover

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

In [3]:
Map = geemap.Map(center=[42.442501697577775, 129.7403904429394], zoom=12)

dataset = ee.ImageCollection("ESA/WorldCover/v100").first()
Map.addLayer(dataset, {'bands': ['Map']}, "Landcover")

ee_class_table = """
Value	Color	Description
10	006400	Trees
20	ffbb22	Shrubland
30	ffff4c	Grassland
40	f096ff	Cropland
50	fa0000	Built-up
60	b4b4b4	Barren / sparse vegetation
70	f0f0f0	Snow and ice
80	0064c8	Open water
90	0096a0	Herbaceous wetland
95	00cf75	Mangroves
100	fae6a0	Moss and lichen
"""

legend_dict = geemap.legend_from_ee(ee_class_table)
Map.add_legend(title="ESA Land Cover", legend_dict=legend_dict)
Map

Map(center=[42.442501697577775, 129.7403904429394], controls=(WidgetControl(options=['position', 'transparent_…